# Recommender System in spark

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('rec').getOrCreate()

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [7]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [21]:
data = spark.read.csv('../datasets/u.data',inferSchema=True,header=True,sep='\t').toDF('user_id', 'item_id', 'rating', 'timestamp')

In [22]:
data.show()

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|      0|    172|     5|881250949|
|      0|    133|     1|881250949|
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
|    298|    474|     4|884182806|
|    115|    265|     2|881171488|
|    253|    465|     5|891628467|
|    305|    451|     3|886324817|
|      6|     86|     3|883603013|
|     62|    257|     2|879372434|
|    286|   1014|     5|879781125|
|    200|    222|     5|876042340|
|    210|     40|     3|891035994|
|    224|     29|     3|888104457|
|    303|    785|     3|879485318|
|    122|    387|     5|879270459|
|    194|    274|     2|879539794|
+-------+-------+------+---------+
only showing top 20 rows



In [23]:
data.describe().show()

+-------+------------------+------------------+------------------+-------------------+
|summary|           user_id|           item_id|            rating|          timestamp|
+-------+------------------+------------------+------------------+-------------------+
|  count|            100002|            100002|            100002|             100002|
|   mean| 462.4755004899902| 425.5246695066099|  3.52984940301194|8.835288059314814E8|
| stddev|266.61977624270713|330.79731330193573|1.1257003701282393|  5343812.460921578|
|    min|                 0|                 1|                 1|          874724710|
|    max|               943|              1682|                 5|          893286638|
+-------+------------------+------------------+------------------+-------------------+



#### Dropping the timestamp column

In [24]:
data = data.drop('timestamp')
data.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [25]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [26]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="item_id", ratingCol="rating")
model = als.fit(training)

Now let's see hwo the model performed!

In [27]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [28]:
predictions.show()

+-------+-------+------+----------+
|user_id|item_id|rating|prediction|
+-------+-------+------+----------+
|     27|    148|     3| 2.0473723|
|    606|    148|     3|  3.767642|
|    663|    148|     4| 3.1720264|
|    435|    148|     3| 3.4015334|
|    164|    148|     5| 4.9801483|
|    919|    148|     3| 3.1183827|
|    923|    148|     4| 3.3922474|
|    891|    148|     5| 4.3217783|
|    423|    148|     3| 3.2256665|
|    870|    148|     2| 2.5004747|
|    757|    148|     4| 2.8417468|
|     84|    148|     4| 3.4418552|
|    532|    148|     5|  3.970731|
|    793|    148|     4|  3.473913|
|     21|    148|     1| 2.0754457|
|     90|    148|     2| 3.2718034|
|    805|    148|     2| 2.9288888|
|    158|    148|     4|   3.15201|
|    276|    148|     3| 2.6996841|
|    186|    148|     4| 3.2206693|
+-------+-------+------+----------+
only showing top 20 rows



In [39]:
predictions.describe().show()

+-------+------------------+-----------------+------------------+----------+
|summary|           user_id|          item_id|            rating|prediction|
+-------+------------------+-----------------+------------------+----------+
|  count|             19940|            19940|             19940|     19940|
|   mean|463.53625877632896|423.9002006018054|3.5353059177532598|       NaN|
| stddev| 268.4097373231257|330.5524957930414|1.1247513867629875|       NaN|
|    min|                 1|                1|                 1| -2.281641|
|    max|               943|             1681|                 5|       NaN|
+-------+------------------+-----------------+------------------+----------+



In [37]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
#Dropping the null prediction values
rmse = evaluator.evaluate(predictions.na.drop())
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.0656292589794112


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [41]:
single_user = test.filter(test['user_id']==11).select(['item_id','user_id'])

In [42]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+-------+
|item_id|user_id|
+-------+-------+
|      9|     11|
|     15|     11|
|     24|     11|
|     28|     11|
|     51|     11|
|     90|     11|
|     97|     11|
|     98|     11|
|    120|     11|
|    135|     11|
|    175|     11|
|    185|     11|
|    204|     11|
|    211|     11|
|    228|     11|
|    260|     11|
|    277|     11|
|    286|     11|
|    300|     11|
|    312|     11|
+-------+-------+
only showing top 20 rows



In [43]:
reccomendations = model.transform(single_user)

In [44]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+-------+----------+
|item_id|user_id|prediction|
+-------+-------+----------+
|    750|     11| 4.7343364|
|    286|     11|  4.181887|
|     15|     11|    4.0924|
|     98|     11|  4.044594|
|    744|     11| 4.0417757|
|      9|     11| 3.9970865|
|    451|     11| 3.9081779|
|    742|     11| 3.8920014|
|     51|     11| 3.8534713|
|     28|     11| 3.8355749|
|    300|     11| 3.8213162|
|     97|     11|  3.694994|
|    549|     11|  3.653762|
|    185|     11| 3.6291761|
|    204|     11| 3.6225276|
|    732|     11| 3.5971987|
|    211|     11|  3.589419|
|    393|     11|  3.491588|
|    405|     11|  3.437232|
|    524|     11| 3.3460574|
+-------+-------+----------+
only showing top 20 rows



`That's it for this recommender system`